In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('raw_data.csv') # изменить путь
df

,idx,text,label
0,10026,[PERSON]\n\nУважаемый [PERSON]!\n\n[ORGANIZATI...,Блок директора по проектированию
1,1005,[ORGANIZATION] инжиниринг общество с ограничен...,Блок деректора по газу
2,1010,[ORGANIZATION] ИНВЕСТ Общество с ограниченной ...,Блок заместителя генерального директора по без...
3,1013,[ORGANIZATION] филиал [OBJECT] имени [PERSON] ...,Блок технического директора
4,10186,[ORGANIZATION] ЗАМЕСТИТЕЛЬ ГЕНЕРАЛЬНОГО ДИРЕКТ...,Блок директора по проектированию
...,...,...,...
1416,7295,[ORGANIZATION] юридический адрес: [LOCATION] О...,Блок технического директора
1417,7296,[ORGANIZATION] сообщает [PERSON] о готовности ...,Блок технического директора
1418,7297,[ORGANIZATION] Юридический адрес: [LOCATION] О...,Блок технического директора
1419,27278G,ООО [НАЗВАНИЕ_КОМПАНИИ] Контактная информация:...,Проект «Трубопроводный транспорт Ещё одного НГКМ»


In [29]:
def low(df):
    for i in range(len(df)):
        df["text"][i] = df["text"][i].lower()
    return df

df = low(df)

C:\Users\k4484\AppData\Local\Temp\ipykernel_24016\2463731649.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["text"][i] = df["text"][i].lower()


# Анализ самых длинных и коротких текстов

In [ ]:
long_text_mask = df['text'].astype(str).str.len() < 410
idx_values = df.loc[long_text_mask, 'idx'].tolist()

idx_values
for i in idx_values:
    # if (len(df[df['idx'] == i]['text'].tolist()[0]) < 400): continue
    print(i, df[df['idx'] == i]['label'].tolist()[0])
    print(df[df['idx'] == i]['text'].tolist()[0])
    print()


1310 Блок заместителя генерального директора по организационным вопросам
[ORGANIZATION] [LOCATION] [DOCUMENT_NUMBER] [CONTACT] Уважаемый [PERSON]! В связи с загрузкой на ВЖК и ВЖГ [OBJECT] [DATE_TIME] основным меню, заказными позициями (выпечка пирогов) и ДНК [OBJECT] с дегустационным столом, персонал столовых не справится одновременно с приготовлением. Прошу вас согласовать перенос проведения ДНК на [OBJECT] на [DATE_TIME]. [PERSON]

4377 Блок директора по строительству
САМОРЕГУЛИРУЕМАЯ ОРГАНИЗАЦИЯ [ORGANIZATION] [LOCATION] [CONTACT] E-mail: [CONTACT], www: [WEBSITE] [ID] [DOCUMENT_NUMBER] Исполнитель [PERSON] [PERSON] Тел. [CONTACT] QR-код на Telegram-канал [ORGANIZATION]

5586 Блок операционного директора
[ORGANIZATION] [ORGANIZATION] [LOCATION] [CONTACT] [DATE_TIME] No[DOCUMENT_NUMBER] [PERSON] [PERSON] Уважаемый [PERSON]! [ORGANIZATION] просит выделить бульдозер для расчистки полигона ТБО [LOCATION]. [PERSON]

5587 Блок директора по строительству
[ORGANIZATION] [ORGANIZATION] [ORG

1310 - норм, можно не удалять
4377 - "Блок директора по строительству", удаляем
5586 - "Блок операционного директора", удаляем
5587 - "Блок директора по строительству", удаляем
5749 - "Блок директора по строительству", удаляем
5758 - "Блок директора по строительству", удаляем
5955 - "Блок технического директора", удаляем
6053 - "Блок директора по мощностям", удаляем
6925 - "Блок директора по мощностям", удаляем
7033 - норм, можно не удалять
7118 - норм, можно не удалять
Итог - ['4377', '5586', '5587', '5749', '5758', '5955', '6053', '6925']
# !Среди текстов длиннее 410 символов при беглом взгляде все нормально 

In [ ]:
def remove_duplicates_instr(text: str, spaces: int = 3) -> str:
    # spaces - через сколько слов происходит разрыв    
    splitted_text = text.split()
    index = 0
    while index < len(splitted_text):
        other_substr = list(filter(lambda x: x[1] == splitted_text[index], [(k, splitted_text[k]) for k in range(index+1, len(splitted_text))]))
        if len(other_substr) == 0:
            index+=1
            continue
        new_other_substr = [other_substr[0]]
        for j in range(1, len(other_substr)):
            if new_other_substr[-1][0] + spaces < other_substr[j][0]:
                break
            new_other_substr.append(other_substr[j])
        for j in new_other_substr:
            splitted_text[j[0]] = ''
        splitted_text = [j for j in splitted_text if j != '']
        index+=1
    return ' '.join(splitted_text)

print(remove_duplicates_instr('fgf  \n hg hg hg hg hg hg hg hg hg hg jj hg kk'))

fgf hg          jj  kk


In [ ]:
def remove_duplicates_with_condition(text: str) -> str:
    if len(text) > 10000:
        return remove_duplicates_instr(text)
    return text

In [ ]:
# df_without_duplicates_in_text = df.copy(deep=True)
# print(123)
# df_without_duplicates_in_text['text'] = df['text'].apply(remove_duplicates_with_condition)

long_text_mask = df['text'].astype(str).str.len() > 10000
idx_values = df.loc[long_text_mask, 'idx'].tolist()

# print(len(idx_values))

# idx_values
for i in range(len(idx_values)):
    print(i+1)
    print(idx_values[i], df[df['idx'] == idx_values[i]]['label'].tolist()[0])
    print(remove_duplicates_instr(df[df['idx'] == idx_values[i]]['text'].tolist()[0]))
    print(len(remove_duplicates_instr(df[df['idx'] == idx_values[i]]['text'].tolist()[0])))
    print()

1
1215 Имущественные вопросы
[organization] приказ [date_time] no [document_number] об утверждении положений по непрофильным активам и порядке отчуждения непрофильных активов в целях приведения локальных нормативных актов группы вопросам управления непрофильными активами соответствии с нормативными актами [organization] приказываю: 1. утвердить положение о комиссии по организаций компаний в новой редакции (приложение 1). 2. [organization] и группы 2). 3. утвердить состав непрофильным активам [organization] в новом составе (приложение 3). 4. признать утратившими силу приказы от [date_time] no [document_number]. 5. распространить действие настоящего приказа на организации компаний [organization], которых компания имеет право прямо или косвенно распоряжаться более 50 голосов 4). 6. контроль исполнения возложить заместителя председателя правления [person] председатель приложение 1 к приказу [organization] от [document_number] положение о комиссии и организаций группы [organization] по общи

1263 Блок заместителя генерального директора по организационным вопросам - удалить
4347 Блок заместителя генерального директора по безопасности - удалить
4447 Блок директора по строительству - удалить
512 Блок деректора по газу - удалить
5602 Блок технического директора - удалить
5682 Блок технического директора - удалить
5831 Блок директора по мощностям - удалить
5909 Блок заместителя генерального директора по организационным вопросам - удалить
6154 Генеральный директор - удалить
6344 Блок технического директора - удалить
7012 Блок деректора по газу - удалить
6666 Блок директора по мощностям - удалить
7047 Блок финансового директора - удалить
6884 Проект сервиса скважин - удалить
6029 Блок заместителя генерального директора по организационным вопросам - удалить/изменить
6526 Блок директора по персоналу - удалить/изменить
6644 Блок директора по мощностям - удалить/изменить
5931 Блок директора по мощностям - удалить/изменить
5980 Блок технического директора - удалить/изменить
7077 Проект «Обустройство объектов Новой нефти» - удалить/изменить !класс представлен только в 14 экземплярах
6571 Проект «Обустройство Интересного лицензионного участка» - удалить/изменить !класс представлен только в 6 экземплярах
6599 Управление землеустроительных работ - удалить/изменить !класс представлен только в 18 экземплярах
['1263', '4347', '4447', '512', '5602', '5682', '5831', '5909', '6154', '6344', '7012', '6666', '7047', '6884']
['6029', '6526', '6644', '5931', '5980']

In [74]:
a = "общество с ограниченной ответственностью [organization] генеральный директор генеральному директору [organization] [person] почтовый адрес: [location] юридический тел.: [contact], e-mail: [contact] огрн [id], инн/кпп [id] [date_time] no [document_number] на от о возмещении абонентской платы уважаемый [person]! между (далее – исполнитель) и [organization] заказчик) заключен договор no оказание транспортных услуг на проекте [object]. следуя процедуре компенсации затрат установку обслуживание системы интеллектуального видеонаблюдения на средствах, прошу возместить стоимость абонентского обслуживания за [date_time] и в размере [financial_data] (двести семьдесят четыре тысячи девятьсот двадцать рублей 00 коп.). приложения: 1. [document_number] от [date_time] информационные услуги 2. счет-фактура no оказанные 3. платежное поручение [document_number] от [date_time] 4. no 5. реестр транспортных средств [organization], оснащенных системой видеоаналитики, за [date_time] 6. 7. расчет суммы возмещения уважением, публичная оферта к заключению договора на информационные услуги договор [organization] [document_number] г. [city] [date_time] с [organization], инн кпп имеющее дальнейшем исполнитель, лице генерального директора [person], действующего на основании устава, одной стороны, общество с ограниченной ответственностью огрн [id], р/с в банке бик к/с [id], заказчик, другой вместе именуемые в дальнейшем стороны, заключили настоящий нижеследующем: условия предоставления соответствии со статьей 435 гк рф являются офертой, адресованной неограниченному множеству заказчиков исполнителем (далее по тексту - договор или оферта). предмет термины и определения: услуги – информационные предоставлению доступов отчетам системы в рамках настоящего договора; оборудование навигационные терминалы, дополнительные датчики, передающие телеметрию с объектов системы. система – программное обеспечение, предоставляющее собственный веб-интерфейс [website], либо альтернативные веб-интерфейсы которых настроены профили объектов, которые используются для формирования отчетов о работе объектов; личный кабинет раздел сайта, где зарегистрированные пользователи могут совершать определенные действия, авторизовавшись в нем. объект (или тс) – транспортное средство, подвижный и стационарный заказчика, профиль которого настроен системе, по которому в систему может передаваться телеметрия от оборудования; эдо электронного документооборота информационная система, которой осуществляется обмен информацией в электронной форме участниками информационного взаимодействия, действующая правилам, установленным владельцем (оператором) эдо, и представляющая собой совокупность программного, аппаратного обеспечения; эцп - электронная цифровая подпись – реквизит документа, предназначенный защиты данного электронного документа подделки, полученный результате криптографического преобразования информации использованием открытого ключа цифровой подписи и позволяющий идентифицировать владельца сертификата подписи, а также установить отсутствие искажения в электронном документе; [website] мобильное приложение расположенного на web-адресе - система обмена значимыми электронными документами заявками заказчика сервисной службы исполнителя электронной посредством сети интернет. работает как управления клиентскими обращениями через приложение, устанавливаемое телефон представителя заказчика, решающая задачу быстрой регистрации, квалификации и реакции на входящую заявку. также, данное решает задачи размещения, отслеживания статуса отправления выполнения заявок заказчика; тариф оказания информационной услуги для одного объекта; тарифная опция – стоимость дополнительной объекта. документ подписан подписью оператор [organization] идентификатор: [id] отправлено [organization], [person], утверждено генеральный директор [date_time] страница 1 bx. no[document_number] от 1.1 исполнитель обязуется предоставить заказчику услуги услуги), заказчик принять и оплатить эти услуги. 1.2 порядок услуг: 1.2.1. оказываются путем предоставления доступа к личному кабинету в системе услуги предоставляются 24 часа сутки, ежедневно, без перерывов, за исключением проведения профилактических ремонтных работ. 1.2.2. подключение и доступ сети интернет заказчиком самостоятельно посредством провайдера интернет-услуг, с которым у договор. 1.2.3. подключает в личном кабинете заказчика согласно представленному списку. 1.2.4. любое время действия договора пользуется услугами подключается к в целях объектов. права и обязанности сторон 2.1 исполнитель обязуется: 2.1.1. обеспечивать работу личного кабинета настоящим договором, круглосуточно 7 дней в неделю, включая выходные праздничные дни, случаев, оговоренных настоящем договоре. 2.1.2. соблюдать конфиденциальность учетных данных заказчика. имеет информации в технического обеспечения для услуг. резервное копирование заказчика целях предотвращения потери не является нарушением конфиденциальности информации 2.2 заказчик обязан: 2.2.1. своевременно, в предусмотренные договором сроки, оплачивать исполнителя. 2.2.2. иметь персональный компьютер (планшет, смартфон) с выходом интернет. 2.2.3. авторские предоставляемые и/или третьими лицами обеспечение и документацию. 2.2.4. письменно уведомлять исполнителя об намерении увеличения/уменьшения количества в до начала расчетного месяца. 2.3 исполнитель право: 2.3.1. течение (одного) дня момента расторжения прекращения срока ограничить (блокировать) доступ к личному кабинету удалить все данные заказчика из возможности восстановления освобождается любых обязательств, связанных с хранением данных заказчика. 2.3.2. по истечении 1 календарного месяца, нарушения заказчиком условий оплаты исполнителю за услуги, с последующим удалением всех личного кабинета без информации. при нарушении стороны заказчика от данных 2.3.3. ссылаясь на заказчика, пользователя системы, в любой форме и на любом носителе (в т.ч. сайте системы любых других сайтах и/или рекламных материалах исполнителя). 2.4 имеет 2.4.1. выбрать своему усмотрению перечень необходимых ему 2.4.2. пролонгировать его действия. 2.4.3. расторгнуть договор досрочно, направив уведомление один месяц до наступления предполагаемой даты прекращения, а оплатив предварительно платежи за указанной даты. 2.4.4. отсутствии возможности применения эдо уведомить этом исполнителю. при вправе использовать способы связи: * электронный стороны обмениваться по электронной почте сообщениями договору, к числу относят договор, приложение нему, соглашения, акты, универсальные передаточные документы, спецификации. [organization], [person] [date_time] сертификат [person], генеральный директор [id] 2 буква б [date_time] счета, претензии и др. электронные образы документов передаваемые по электронной почте, признаются сторонами, имеющими юридическую силу; - на бумажном носителе. в этом случае все созданные рамках настоящего договора документы направляются нарочным (курьерской службой) или почтой россии письмом с описью вложений адресам, указанным сторонами. письмо считается доставленным в момент вручения адресату, но позднее чем 10-ый календарный день момента отправления. срок действия 3.1 заключенным с подписания и действует до включительно по желанию из сторон. если не позднее, 10 (десять) рабочих до даты прекращения договора ни одна письменно заявит о расторжении, настоящий договор автоматически продлевается на следующий год. 3.2 окончание срока действия досрочное расторжение настоящего влечет за прекращение обязательств сторон нему. либо договора не освобождает от исполнения по остались неисполненными или вступают силу связи прекращением настоящего которые условиям договора, сохраняют свое действие после его прекращения. окончание срока действия досрочное расторжение настоящего договора стороны ответственности ненадлежащее исполнение ранее принятых на себя обязательств, также за последствия они имели место в период договора. услуг оплаты 4.1 стоимость настоящему договору определяется зависимости от объема услуг и подключенных к тому или иному тарифу, стоимости тарифа. тарифов [organization] указан в приложении no договору. 4.2 количество тарифицируемых объектов согласно перечню объектов, утвержденных в к настоящему и 4.3 каждый месяц, не 10-го числа, исполнителем создается счет (клиентом) оказываемых текущем месяце, тарифам предоставление услуг, тарифным опционам количеству получение счета в виде признается сторонами достаточным для оплаты услуг. 4.4 услуги оплачиваются на основании условий оплаты, указанных приложении no 1, выставленного исполнителем, позднее 15-го числа текущего месяца перечисления денежных средств расчетный исполнителя, указанный в разделе 8 настоящего датой услуг дата зачисления на счет 4.5 заказчик самостоятельно следит своевременностью внесения платежей. оплаты информационных конец отчетного периода, исполнитель одностороннем порядке приостановить информации по объектам в личном кабинете, а направить заказчику предстоящей блокировке и последующем удалении профилей объектов. удаление производится не ранее, чем 3 (трех) месяцев с отправки такого уведомления. освобождается связанных хранением данных ограничения удаления из кабинета. во избежание данных в кабинете за предыдущий пользования, обязан исполнителя о своих намерениях получить окончания периода. 4.6 исполнитель вправе изменить размер предоставляемых по договору, в адрес заказчика направляется соответствующее уведомление не чем 1 даты изменения стоимости изменение услуг распространяется на оплаченные моменту 4.7 выставление упд ежемесячно не позднее следующего за отчетным, стоимости услуг, оказанных отчетном месяце. 4.8 платеж настоящему договору должен совершаться заказчиком отдельным платежным поручением с обязательной ссылкой на дату номер договора. по совершаются полностью удержаний и вычетов, также без предъявления заказчиком общество с ограниченной ответственностью [organization] генеральный директор генеральному директору [organization] [person] почтовый адрес: [location] юридический тел.: [contact], e-mail: [contact] огрн [id], инн/кпп [id] [date_time] no [document_number] на от о возмещении абонентской платы уважаемый [person]! между (далее – исполнитель) и [organization] заказчик) заключен договор no оказание транспортных услуг на проекте [object]. следуя процедуре компенсации затрат установку обслуживание системы интеллектуального видеонаблюдения на средствах, прошу возместить стоимость абонентского обслуживания за [date_time] и в размере [financial_data] (двести семьдесят четыре тысячи девятьсот двадцать рублей 00 коп.). приложения: 1. [document_number] от [date_time] информационные услуги 2. счет-фактура no оказанные 3. платежное поручение [document_number] от [date_time] 4. no 5. реестр транспортных средств [organization], оснащенных системой видеоаналитики, за [date_time] 6. 7. расчет суммы возмещения уважением, публичная оферта к заключению договора на информационные услуги договор [organization] [document_number] г. [city] [date_time] с [organization], инн кпп имеющее дальнейшем исполнитель, лице генерального директора [person], действующего на основании устава, одной стороны, общество с ограниченной ответственностью огрн [id], р/с в банке бик к/с [id], заказчик, другой вместе именуемые в дальнейшем стороны, заключили настоящий нижеследующем: условия предоставления соответствии со статьей 435 гк рф являются офертой, адресованной неограниченному множеству заказчиков исполнителем (далее по тексту - договор или оферта). предмет термины и определения: услуги – информационные предоставлению доступов отчетам системы в рамках настоящего договора; оборудование навигационные терминалы, дополнительные датчики, передающие телеметрию с объектов системы. система – программное обеспечение, предоставляющее собственный веб-интерфейс [website], либо альтернативные веб-интерфейсы которых настроены профили объектов, которые используются для формирования отчетов о работе объектов; личный кабинет раздел сайта, где зарегистрированные пользователи могут совершать определенные действия, авторизовавшись в нем. объект (или тс) – транспортное средство, подвижный и стационарный заказчика, профиль которого настроен системе, по которому в систему может передаваться телеметрия от оборудования; эдо электронного документооборота информационная система, которой осуществляется обмен информацией в электронной форме участниками информационного взаимодействия, действующая правилам, установленным владельцем (оператором) эдо, и представляющая собой совокупность программного, аппаратного обеспечения; эцп - электронная цифровая подпись – реквизит документа, предназначенный защиты данного электронного документа подделки, полученный результате криптографического преобразования информации использованием открытого ключа цифровой подписи и позволяющий идентифицировать владельца сертификата подписи, а также установить отсутствие искажения в электронном документе; [website] мобильное приложение расположенного на web-адресе - система обмена значимыми электронными документами заявками заказчика сервисной службы исполнителя электронной посредством сети интернет. работает как управления клиентскими обращениями через приложение, устанавливаемое телефон представителя заказчика, решающая задачу быстрой регистрации, квалификации и реакции на входящую заявку. также, данное решает задачи размещения, отслеживания статуса отправления выполнения заявок заказчика; тариф оказания информационной услуги для одного объекта; тарифная опция – стоимость дополнительной объекта. документ подписан подписью оператор [organization] идентификатор: [id] отправлено [organization], [person], утверждено генеральный директор [date_time] страница 1 bx. no[document_number] от 1.1 исполнитель обязуется предоставить заказчику услуги услуги), заказчик принять и оплатить эти услуги. 1.2 порядок услуг: 1.2.1. оказываются путем предоставления доступа к личному кабинету в системе услуги предоставляются 24 часа сутки, ежедневно, без перерывов, за исключением проведения профилактических ремонтных работ. 1.2.2. подключение и доступ сети интернет заказчиком самостоятельно посредством провайдера интернет-услуг, с которым у договор. 1.2.3. подключает в личном кабинете заказчика согласно представленному списку. 1.2.4. любое время действия договора пользуется услугами подключается к в целях объектов. права и обязанности сторон 2.1 исполнитель обязуется: 2.1.1. обеспечивать работу личного кабинета настоящим договором, круглосуточно 7 дней в неделю, включая выходные праздничные дни, случаев, оговоренных настоящем договоре. 2.1.2. соблюдать конфиденциальность учетных данных заказчика. имеет информации в технического обеспечения для услуг. резервное копирование заказчика целях предотвращения потери не является нарушением конфиденциальности информации 2.2 заказчик обязан: 2.2.1. своевременно, в предусмотренные договором сроки, оплачивать исполнителя. 2.2.2. иметь персональный компьютер (планшет, смартфон) с выходом интернет. 2.2.3. авторские предоставляемые и/или третьими лицами обеспечение и документацию. 2.2.4. письменно уведомлять исполнителя об намерении увеличения/уменьшения количества в до начала расчетного месяца. 2.3 исполнитель право: 2.3.1. течение (одного) дня момента расторжения прекращения срока ограничить (блокировать) доступ к личному кабинету удалить все данные заказчика из возможности восстановления освобождается любых обязательств, связанных с хранением данных заказчика. 2.3.2. по истечении 1 календарного месяца, нарушения заказчиком условий оплаты исполнителю за услуги, с последующим удалением всех личного кабинета без информации. при нарушении стороны заказчика от данных 2.3.3. ссылаясь на заказчика, пользователя системы, в любой форме и на любом носителе (в т.ч. сайте системы любых других сайтах и/или рекламных материалах исполнителя). 2.4 имеет 2.4.1. выбрать своему усмотрению перечень необходимых ему 2.4.2. пролонгировать его действия. 2.4.3. расторгнуть договор досрочно, направив уведомление один месяц до наступления предполагаемой даты прекращения, а оплатив предварительно платежи за указанной даты. 2.4.4. отсутствии возможности применения эдо уведомить этом исполнителю. при вправе использовать способы связи: * электронный стороны обмениваться по электронной почте сообщениями договору, к числу относят договор, приложение нему, соглашения, акты, универсальные передаточные документы, спецификации. [organization], [person] [date_time] сертификат [person], генеральный директор [id] 2 буква б [date_time] счета, претензии и др. электронные образы документов передаваемые по электронной почте, признаются сторонами, имеющими юридическую силу; - на бумажном носителе. в этом случае все созданные рамках настоящего договора документы направляются нарочным (курьерской службой) или почтой россии письмом с описью вложений адресам, указанным сторонами. письмо считается доставленным в момент вручения адресату, но позднее чем 10-ый календарный день момента отправления. срок действия 3.1 заключенным с подписания и действует до включительно по желанию из сторон. если не позднее, 10 (десять) рабочих до даты прекращения договора ни одна письменно заявит о расторжении, настоящий договор автоматически продлевается на следующий год. 3.2 окончание срока действия досрочное расторжение настоящего влечет за прекращение обязательств сторон нему. либо договора не освобождает от исполнения по остались неисполненными или вступают силу связи прекращением настоящего которые условиям договора, сохраняют свое действие после его прекращения. окончание срока действия досрочное расторжение настоящего договора стороны ответственности ненадлежащее исполнение ранее принятых на себя обязательств, также за последствия они имели место в период договора. услуг оплаты 4.1 стоимость настоящему договору определяется зависимости от объема услуг и подключенных к тому или иному тарифу, стоимости тарифа. тарифов [organization] указан в приложении no договору. 4.2 количество тарифицируемых объектов согласно перечню объектов, утвержденных в к настоящему и 4.3 каждый месяц, не 10-го числа, исполнителем создается счет (клиентом) оказываемых текущем месяце, тарифам предоставление услуг, тарифным опционам количеству получение счета в виде признается сторонами достаточным для оплаты услуг. 4.4 услуги оплачиваются на основании условий оплаты, указанных приложении no 1, выставленного исполнителем, позднее 15-го числа текущего месяца перечисления денежных средств расчетный исполнителя, указанный в разделе 8 настоящего датой услуг дата зачисления на счет 4.5 заказчик самостоятельно следит своевременностью внесения платежей. оплаты информационных конец отчетного периода, исполнитель одностороннем порядке приостановить информации по объектам в личном кабинете, а направить заказчику предстоящей блокировке и последующем удалении профилей объектов. удаление производится не ранее, чем 3 (трех) месяцев с отправки такого уведомления. освобождается связанных хранением данных ограничения удаления из кабинета. во избежание данных в кабинете за предыдущий пользования, обязан исполнителя о своих намерениях получить окончания периода. 4.6 исполнитель вправе изменить размер предоставляемых по договору, в адрес заказчика направляется соответствующее уведомление не чем 1 даты изменения стоимости изменение услуг распространяется на оплаченные моменту 4.7 выставление упд ежемесячно не позднее следующего за отчетным, стоимости услуг, оказанных отчетном месяце. 4.8 платеж настоящему договору должен совершаться заказчиком отдельным платежным поручением с обязательной ссылкой на дату номер договора. по совершаются полностью удержаний и вычетов, также без предъявления заказчиком общество с ограниченной ответственностью [organization] генеральный директор генеральному директору [organization] [person] почтовый адрес: [location] юридический тел.: [contact], e-mail: [contact] огрн [id], инн/кпп [id] [date_time] no [document_number] на от о возмещении абонентской платы уважаемый [person]! между (далее – исполнитель) и [organization] заказчик) заключен договор no оказание транспортных услуг на проекте [object]. следуя процедуре компенсации затрат установку обслуживание системы интеллектуального видеонаблюдения на средствах, прошу возместить стоимость абонентского обслуживания за [date_time] и в размере [financial_data] (двести семьдесят четыре тысячи девятьсот двадцать рублей 00 коп.). приложения: 1. [document_number] от [date_time] информационные услуги 2. счет-фактура no оказанные 3. платежное поручение [document_number] от [date_time] 4. no 5. реестр транспортных средств [organization], оснащенных системой видеоаналитики, за [date_time] 6. 7. расчет суммы возмещения уважением, публичная оферта к заключению договора на информационные услуги договор [organization] [document_number] г. [city] [date_time] с [organization], инн кпп имеющее дальнейшем исполнитель, лице генерального директора [person], действующего на основании устава, одной стороны, общество с ограниченной ответственностью огрн [id], р/с в банке бик к/с [id], заказчик, другой вместе именуемые в дальнейшем стороны, заключили настоящий нижеследующем: условия предоставления соответствии со статьей 435 гк рф являются офертой, адресованной неограниченному множеству заказчиков исполнителем (далее по тексту - договор или оферта). предмет термины и определения: услуги – информационные предоставлению доступов отчетам системы в рамках настоящего договора; оборудование навигационные терминалы, дополнительные датчики, передающие телеметрию с объектов системы. система – программное обеспечение, предоставляющее собственный веб-интерфейс [website], либо альтернативные веб-интерфейсы которых настроены профили объектов, которые используются для формирования отчетов о работе объектов; личный кабинет раздел сайта, где зарегистрированные пользователи могут совершать определенные действия, авторизовавшись в нем. объект (или тс) – транспортное средство, подвижный и стационарный заказчика, профиль которого настроен системе, по которому в систему может передаваться телеметрия от оборудования; эдо электронного документооборота информационная система, которой осуществляется обмен информацией в электронной форме участниками информационного взаимодействия, действующая правилам, установленным владельцем (оператором) эдо, и представляющая собой совокупность программного, аппаратного обеспечения; эцп - электронная цифровая подпись – реквизит документа, предназначенный защиты данного электронного документа подделки, полученный результате криптографического преобразования информации использованием открытого ключа цифровой подписи и позволяющий идентифицировать владельца сертификата подписи, а также установить отсутствие искажения в электронном документе; [website] мобильное приложение расположенного на web-адресе - система обмена значимыми электронными документами заявками заказчика сервисной службы исполнителя электронной посредством сети интернет. работает как управления клиентскими обращениями через приложение, устанавливаемое телефон представителя заказчика, решающая задачу быстрой регистрации, квалификации и реакции на входящую заявку. также, данное решает задачи размещения, отслеживания статуса отправления выполнения заявок заказчика; тариф оказания информационной услуги для одного объекта; тарифная опция – стоимость дополнительной объекта. документ подписан подписью оператор [organization] идентификатор: [id] отправлено [organization], [person], утверждено генеральный директор [date_time] страница 1 bx. no[document_number] от 1.1 исполнитель обязуется предоставить заказчику услуги услуги), заказчик принять и оплатить эти услуги. 1.2 порядок услуг: 1.2.1. оказываются путем предоставления доступа к личному кабинету в системе услуги предоставляются 24 часа сутки, ежедневно, без перерывов, за исключением проведения профилактических ремонтных работ. 1.2.2. подключение и доступ сети интернет заказчиком самостоятельно посредством провайдера интернет-услуг, с которым у договор. 1.2.3. подключает в личном кабинете заказчика согласно представленному списку. 1.2.4. любое время действия договора пользуется услугами подключается к в целях объектов. права и обязанности сторон 2.1 исполнитель обязуется: 2.1.1. обеспечивать работу личного кабинета настоящим договором, круглосуточно 7 дней в неделю, включая выходные праздничные дни, случаев, оговоренных настоящем договоре. 2.1.2. соблюдать конфиденциальность учетных данных заказчика. имеет информации в технического обеспечения для услуг. резервное копирование заказчика целях предотвращения потери не является нарушением конфиденциальности информации 2.2 заказчик обязан: 2.2.1. своевременно, в предусмотренные договором сроки, оплачивать исполнителя. 2.2.2. иметь персональный компьютер (планшет, смартфон) с выходом интернет. 2.2.3. авторские предоставляемые и/или третьими лицами обеспечение и документацию. 2.2.4. письменно уведомлять исполнителя об намерении увеличения/уменьшения количества в до начала расчетного месяца. 2.3 исполнитель право: 2.3.1. течение (одного) дня момента расторжения прекращения срока ограничить (блокировать) доступ к личному кабинету удалить все данные заказчика из возможности восстановления освобождается любых обязательств, связанных с хранением данных заказчика. 2.3.2. по истечении 1 календарного месяца, нарушения заказчиком условий оплаты исполнителю за услуги, с последующим удалением всех личного кабинета без информации. при нарушении стороны заказчика от данных 2.3.3. ссылаясь на заказчика, пользователя системы, в любой форме и на любом носителе (в т.ч. сайте системы любых других сайтах и/или рекламных материалах исполнителя). 2.4 имеет 2.4.1. выбрать своему усмотрению перечень необходимых ему 2.4.2. пролонгировать его действия. 2.4.3. расторгнуть договор досрочно, направив уведомление один месяц до наступления предполагаемой даты прекращения, а оплатив предварительно платежи за указанной даты. 2.4.4. отсутствии возможности применения эдо уведомить этом исполнителю. при вправе использовать способы связи: * электронный стороны обмениваться по электронной почте сообщениями договору, к числу относят договор, приложение нему, соглашения, акты, универсальные передаточные документы, спецификации. [organization], [person] [date_time] сертификат [person], генеральный директор [id] 2 буква б [date_time] счета, претензии и др. электронные образы документов передаваемые по электронной почте, признаются сторонами, имеющими юридическую силу; - на бумажном носителе. в этом случае все созданные рамках настоящего договора документы направляются нарочным (курьерской службой) или почтой россии письмом с описью вложений адресам, указанным сторонами. письмо считается доставленным в момент вручения адресату, но позднее чем 10-ый календарный день момента отправления. срок действия 3.1 заключенным с подписания и действует до включительно по желанию из сторон. если не позднее, 10 (десять) рабочих до даты прекращения договора ни одна письменно заявит о расторжении, настоящий договор автоматически продлевается на следующий год. 3.2 окончание срока действия досрочное расторжение настоящего влечет за прекращение обязательств сторон нему. либо договора не освобождает от исполнения по остались неисполненными или вступают силу связи прекращением настоящего которые условиям договора, сохраняют свое действие после его прекращения. окончание срока действия досрочное расторжение настоящего договора стороны ответственности ненадлежащее исполнение ранее принятых на себя обязательств, также за последствия они имели место в период договора. услуг оплаты 4.1 стоимость настоящему договору определяется зависимости от объема услуг и подключенных к тому или иному тарифу, стоимости тарифа. тарифов [organization] указан в приложении no договору. 4.2 количество тарифицируемых объектов согласно перечню объектов, утвержденных в к настоящему и 4.3 каждый месяц, не 10-го числа, исполнителем создается счет (клиентом) оказываемых текущем месяце, тарифам предоставление услуг, тарифным опционам количеству получение счета в виде признается сторонами достаточным для оплаты услуг. 4.4 услуги оплачиваются на основании условий оплаты, указанных приложении no 1, выставленного исполнителем, позднее 15-го числа текущего месяца перечисления денежных средств расчетный исполнителя, указанный в разделе 8 настоящего датой услуг дата зачисления на счет 4.5 заказчик самостоятельно следит своевременностью внесения платежей. оплаты информационных конец отчетного периода, исполнитель одностороннем порядке приостановить информации по объектам в личном кабинете, а направить заказчику предстоящей блокировке и последующем удалении профилей объектов. удаление производится не ранее, чем 3 (трех) месяцев с отправки такого уведомления. освобождается связанных хранением данных ограничения удаления из кабинета. во избежание данных в кабинете за предыдущий пользования, обязан исполнителя о своих намерениях получить окончания периода. 4.6 исполнитель вправе изменить размер предоставляемых по договору, в адрес заказчика направляется соответствующее уведомление не чем 1 даты изменения стоимости изменение услуг распространяется на оплаченные моменту 4.7 выставление упд ежемесячно не позднее следующего за отчетным, стоимости услуг, оказанных отчетном месяце. 4.8 платеж настоящему договору должен совершаться заказчиком отдельным платежным поручением с обязательной ссылкой на дату номер договора. по совершаются полностью удержаний и вычетов, также без предъявления заказчиком общество с ограниченной ответственностью [organization] генеральный директор генеральному директору [organization] [person] почтовый адрес: [location] юридический тел.: [contact], e-mail: [contact] огрн [id], инн/кпп [id] [date_time] no [document_number] на от о возмещении абонентской платы уважаемый [person]! между (далее – исполнитель) и [organization] заказчик) заключен договор no оказание транспортных услуг на проекте [object]. следуя процедуре компенсации затрат установку обслуживание системы интеллектуального видеонаблюдения на средствах, прошу возместить стоимость абонентского обслуживания за [date_time] и в размере [financial_data] (двести семьдесят четыре тысячи девятьсот двадцать рублей 00 коп.). приложения: 1. [document_number] от [date_time] информационные услуги 2. счет-фактура no оказанные 3. платежное поручение [document_number] от [date_time] 4. no 5. реестр транспортных средств [organization], оснащенных системой видеоаналитики, за [date_time] 6. 7. расчет суммы возмещения уважением, публичная оферта к заключению договора на информационные услуги договор [organization] [document_number] г. [city] [date_time] с [organization], инн кпп имеющее дальнейшем исполнитель, лице генерального директора [person], действующего на основании устава, одной стороны, общество с ограниченной ответственностью огрн [id], р/с в банке бик к/с [id], заказчик, другой вместе именуемые в дальнейшем стороны, заключили настоящий нижеследующем: условия предоставления соответствии со статьей 435 гк рф являются офертой, адресованной неограниченному множеству заказчиков исполнителем (далее по тексту - договор или оферта). предмет термины и определения: услуги – информационные предоставлению доступов отчетам системы в рамках настоящего договора; оборудование навигационные терминалы, дополнительные датчики, передающие телеметрию с объектов системы. система – программное обеспечение, предоставляющее собственный веб-интерфейс [website], либо альтернативные веб-интерфейсы которых настроены профили объектов, которые используются для формирования отчетов о работе объектов; личный кабинет раздел сайта, где зарегистрированные пользователи могут совершать определенные действия, авторизовавшись в нем. объект (или тс) – транспортное средство, подвижный и стационарный заказчика, профиль которого настроен системе, по которому в систему может передаваться телеметрия от оборудования; эдо электронного документооборота информационная система, которой осуществляется обмен информацией в электронной форме участниками информационного взаимодействия, действующая правилам, установленным владельцем (оператором) эдо, и представляющая собой совокупность программного, аппаратного обеспечения; эцп - электронная цифровая подпись – реквизит документа, предназначенный защиты данного электронного документа подделки, полученный результате криптографического преобразования информации использованием открытого ключа цифровой подписи и позволяющий идентифицировать владельца сертификата подписи, а также установить отсутствие искажения в электронном документе; [website] мобильное приложение расположенного на web-адресе - система обмена значимыми электронными документами заявками заказчика сервисной службы исполнителя электронной посредством сети интернет. работает как управления клиентскими обращениями через приложение, устанавливаемое телефон представителя заказчика, решающая задачу быстрой регистрации, квалификации и реакции на входящую заявку. также, данное решает задачи размещения, отслеживания статуса отправления выполнения заявок заказчика; тариф оказания информационной услуги для одного объекта; тарифная опция – стоимость дополнительной объекта. документ подписан подписью оператор [organization] идентификатор: [id] отправлено [organization], [person], утверждено генеральный директор [date_time] страница 1 bx. no[document_number] от 1.1 исполнитель обязуется предоставить заказчику услуги услуги), заказчик принять и оплатить эти услуги. 1.2 порядок услуг: 1.2.1. оказываются путем предоставления доступа к личному кабинету в системе услуги предоставляются 24 часа сутки, ежедневно, без перерывов, за исключением проведения профилактических ремонтных работ. 1.2.2. подключение и доступ сети интернет заказчиком самостоятельно посредством провайдера интернет-услуг, с которым у договор. 1.2.3. подключает в личном кабинете заказчика согласно представленному списку. 1.2.4. любое время действия договора пользуется услугами подключается к в целях объектов. права и обязанности сторон 2.1 исполнитель обязуется: 2.1.1. обеспечивать работу личного кабинета настоящим договором, круглосуточно 7 дней в неделю, включая выходные праздничные дни, случаев, оговоренных настоящем договоре. 2.1.2. соблюдать конфиденциальность учетных данных заказчика. имеет информации в технического обеспечения для услуг. резервное копирование заказчика целях предотвращения потери не является нарушением конфиденциальности информации 2.2 заказчик обязан: 2.2.1. своевременно, в предусмотренные договором сроки, оплачивать исполнителя. 2.2.2. иметь персональный компьютер (планшет, смартфон) с выходом интернет. 2.2.3. авторские предоставляемые и/или третьими лицами обеспечение и документацию. 2.2.4. письменно уведомлять исполнителя об намерении увеличения/уменьшения количества в до начала расчетного месяца. 2.3 исполнитель право: 2.3.1. течение (одного) дня момента расторжения прекращения срока ограничить (блокировать) доступ к личному кабинету удалить все данные заказчика из возможности восстановления освобождается любых обязательств, связанных с хранением данных заказчика. 2.3.2. по истечении 1 календарного месяца, нарушения заказчиком условий оплаты исполнителю за услуги, с последующим удалением всех личного кабинета без информации. при нарушении стороны заказчика от данных 2.3.3. ссылаясь на заказчика, пользователя системы, в любой форме и на любом носителе (в т.ч. сайте системы любых других сайтах и/или рекламных материалах исполнителя). 2.4 имеет 2.4.1. выбрать своему усмотрению перечень необходимых ему 2.4.2. пролонгировать его действия. 2.4.3. расторгнуть договор досрочно, направив уведомление один месяц до наступления предполагаемой даты прекращения, а оплатив предварительно платежи за указанной даты. 2.4.4. отсутствии возможности применения эдо уведомить этом исполнителю. при вправе использовать способы связи: * электронный стороны обмениваться по электронной почте сообщениями договору, к числу относят договор, приложение нему, соглашения, акты, универсальные передаточные документы, спецификации. [organization], [person] [date_time] сертификат [person], генеральный директор [id] 2 буква б [date_time] счета, претензии и др. электронные образы документов передаваемые по электронной почте, признаются сторонами, имеющими юридическую силу; - на бумажном носителе. в этом случае все созданные рамках настоящего договора документы направляются нарочным (курьерской службой) или почтой россии письмом с описью вложений адресам, указанным сторонами. письмо считается доставленным в момент вручения адресату, но позднее чем 10-ый календарный день момента отправления. срок действия 3.1 заключенным с подписания и действует до включительно по желанию из сторон. если не позднее, 10 (десять) рабочих до даты прекращения договора ни одна письменно заявит о расторжении, настоящий договор автоматически продлевается на следующий год. 3.2 окончание срока действия досрочное расторжение настоящего влечет за прекращение обязательств сторон нему. либо договора не освобождает от исполнения по остались неисполненными или вступают силу связи прекращением настоящего которые условиям договора, сохраняют свое действие после его прекращения. окончание срока действия досрочное расторжение настоящего договора стороны ответственности ненадлежащее исполнение ранее принятых на себя обязательств, также за последствия они имели место в период договора. услуг оплаты 4.1 стоимость настоящему договору определяется зависимости от объема услуг и подключенных к тому или иному тарифу, стоимости тарифа. тарифов [organization] указан в приложении no договору. 4.2 количество тарифицируемых объектов согласно перечню объектов, утвержденных в к настоящему и 4.3 каждый месяц, не 10-го числа, исполнителем создается счет (клиентом) оказываемых текущем месяце, тарифам предоставление услуг, тарифным опционам количеству получение счета в виде признается сторонами достаточным для оплаты услуг. 4.4 услуги оплачиваются на основании условий оплаты, указанных приложении no 1, выставленного исполнителем, позднее 15-го числа текущего месяца перечисления денежных средств расчетный исполнителя, указанный в разделе 8 настоящего датой услуг дата зачисления на счет 4.5 заказчик самостоятельно следит своевременностью внесения платежей. оплаты информационных конец отчетного периода, исполнитель одностороннем порядке приостановить информации по объектам в личном кабинете, а направить заказчику предстоящей блокировке и последующем удалении профилей объектов. удаление производится не ранее, чем 3 (трех) месяцев с отправки такого уведомления. освобождается связанных хранением данных ограничения удаления из кабинета. во избежание данных в кабинете за предыдущий пользования, обязан исполнителя о своих намерениях получить окончания периода. 4.6 исполнитель вправе изменить размер предоставляемых по договору, в адрес заказчика направляется соответствующее уведомление не чем 1 даты изменения стоимости изменение услуг распространяется на оплаченные моменту 4.7 выставление упд ежемесячно не позднее следующего за отчетным, стоимости услуг, оказанных отчетном месяце. 4.8 платеж настоящему договору должен совершаться заказчиком отдельным платежным поручением с обязательной ссылкой на дату номер договора. по совершаются полностью удержаний и вычетов, также без предъявления заказчиком"
print(len(a))
print(len(remove_duplicates_instr(a)))

39167
21887
